# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto
> *Notebook Created by* **Ezra Witt**

> # `Part 1 ~ Gathering the Data From Wikipedia`

Before we get the data and start exploring it, let's download all the dependencies that we will need for part 1

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
print('Libraries imported.')

Libraries imported.


Since the dataset was readily availible we will use a Wikipedia Page to Find the Necessary Data

In [2]:
### Use Beautiful Soup to scrape the data from the wikipedia page table

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
soup = bs(result.content, 'html.parser')
table = soup.table ### using the inspect feature on the wiki page I was able to find the class as table

Let's take a quick look at the data.

In [13]:
### soup.table ### view the table from the wikipedia page

Now let's wrangle and clean the data so we can turn it into a DataFrame

In [14]:
### wrangle the data and clean it

table = soup.find('table') ### find the table
trs = table.find_all('tr') ### find all the table rows
rows = [] ### convert the row data into string
for tr in trs:
    i = tr.find_all('td') ### the specific cell data
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip() ### copy of the string with trailing characters removed for first column
    borough = row[1].text.rstrip() ### copy of the string with trailing characters removed for next column
    neighborhood = row[2].text.rstrip() ### copy of the string with trailing characters removed for last column
    if borough != 'Not assigned': ### Ignoring cells with a borough that is 'Not assigned'
        if neighborhood == 'Not assigned':
            neighborhood = borough ### if neighborhood name is not listed assigning the borough name
        lst.append([postalcode, borough, neighborhood]) ### adding to be columns

### lst

#### Tranform the data into a *pandas* dataframe by using the data nested in Python dictionaries

In [5]:
cols = ['PostalCode', 'Borough', 'Neighborhood'] ### naming the columns
neighborhoods = pd.DataFrame(lst, columns=cols) ### creating the dataframe

Let's explore our DataFrame

In [8]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Last, let's explore the shape of our data

In [7]:
neighborhoods.shape

(103, 3)

---

> # `Part 2 ~ Assigning the Longitude and Latitude`
>> **After Multiple Attempts I was able to install Geocoder but was unable to get longitude/latitude for locations**

---

#### Import and use the csv file to create a DataFrame of the latitude and longitude values of Toronto Postal Codes

In [9]:
df_ll = pd.read_csv('http://cocl.us/Geospatial_data') ### create Longitude & Latitude (ll) dataframe by using pandas to read csv
df_ll.columns = ['PostalCode','Latitude','Longitude'] ### name the incoming columns

#### view the first five to verify format

In [10]:
df_ll.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Use the Longitude and Latitude DataFrame and merge it into our Toronto Neighborhood DataFrame

In [11]:
Toronto_neighborhoods = pd.merge(neighborhoods, df_ll[['PostalCode','Latitude','Longitude']], on='PostalCode')
Toronto_neighborhoods = Toronto_neighborhoods.sort_values('PostalCode')

Let's view the merged Data Frame

In [12]:
Toronto_neighborhoods

,PostalCode,Borough,Neighborhood,Latitude,Longitude
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
18,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
22,M1G,Scarborough,Woburn,43.770992,-79.216917
26,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
32,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
38,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
44,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
51,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
58,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


> # `Part 3 ~ Exploring and Creating a Cluster Map`

#### Before we get the data and start exploring it, 

let's download all the dependencies that we will need for part 3

In [ ]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

In [ ]:
# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't previously installed
import folium # map rendering library

In [ ]:
# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't installed yet
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

### Use geopy library to get the latitude and longitude values of Toronto

In [ ]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

### Create a map of Toronto with neighborhoods superimposed on top.

In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11.25)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_neighborhoods['Latitude'], Toronto_neighborhoods['Longitude'], Toronto_neighborhoods['Borough'], Toronto_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Define Foursquare Creditials and Version in @hidden_cell

In [ ]:
# The code was removed by Watson Studio for sharing.

### We'll start by exploring the neighborhoods in Toronto by creating a function to run the process for all neighborhoods in Toronto

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Now we write the code to run the function and create a new DataFrame called Toronto_Venues

In [ ]:
Toronto_venues = getNearbyVenues(names=Toronto_neighborhoods['Neighborhood'],
                                   latitudes=Toronto_neighborhoods['Latitude'],
                                   longitudes=Toronto_neighborhoods['Longitude'])

### Let's check how many venues were returned for each neighborhood

In [ ]:
Toronto_venues.groupby('Neighborhood').count()

### Let's see how many unique categories are included in the set

In [ ]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

## `Analyzing the neighborhoods`

In [ ]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

### Let's find each neighborhood's top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

## Create a *pandas* dataframe with the data

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## This time let's find the top 10 venues for each neighborhood

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Toronto_venues_sorted = pd.DataFrame(columns=columns)
Toronto_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    Toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

Toronto_venues_sorted.head()

## `Clustering the Neighborhoods`
#### Run *k-means* to cluster neighborhoods into 5 clusters

In [ ]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

### Creating the dataframe that includes the cluster as well as the top 10 ventues for each neighborhood

In [ ]:
# add clustering labels
Toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(Toronto_venues_sorted.set_index('Neighborhood'), on='Neighborhood').dropna()
Toronto_merged = Toronto_merged.sort_values('Borough')
Toronto_merged.head() # check the last columns!

### `Lastly, let's visualize the resulting clusters`

In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['PostalCode'], Toronto_merged['Borough'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color = rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## `Let's Examine the Resulting Clusters`
### Here you can see the venues that were used to create the clusters

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

#### The first cluster was created by neighborhoods where the parks, playgrounds, and pools are the top venues

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

### The second cluster was created by neighborhoods where food is one of the top venues

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

### The third cluster is by neighborhoods where Sports Fields, Retail, and Industrial were top venues

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

### The fourth cluster is by neighborhoods where parks and Sushi were top venues

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

### The last cluster contains the outlier of our data set, the one that is least like the others.